In [13]:
import os

import warnings
from sklearnex import patch_sklearn
import numpy as np
import pandas as pd
from category_encoders import (
    BackwardDifferenceEncoder,
    BaseNEncoder,
    BinaryEncoder,
    CatBoostEncoder,
    CountEncoder,
    GLMMEncoder,
    HelmertEncoder,
    JamesSteinEncoder,
    LeaveOneOutEncoder,
    MEstimateEncoder,
    SummaryEncoder,
    TargetEncoder,
    WOEEncoder,
)
from sklearn import set_config
from sklearn.base import clone as model_clone
from sklearn.cluster import *
from sklearn.compose import *
from sklearn.cross_decomposition import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.gaussian_process import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.multioutput import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.kernel_approximation import *
from sklearn.svm import *
from sklearn.tree import *
from sklearn.utils import *
from sklearn.dummy import *
from sklearn.semi_supervised import *
from sklearn.discriminant_analysis import *
from sklearn.covariance import *
from collections import Counter
import sklearn
from xgboost import XGBClassifier, XGBRFClassifier

from sklearn.calibration import *
import joblib

pd.options.display.max_columns = 50
set_config(display="diagram")
warnings.filterwarnings("ignore")
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import InstanceHardnessThreshold
from imblearn.over_sampling import ADASYN,SMOTE,RandomOverSampler,SVMSMOTE,SMOTENC,SMOTEN,BorderlineSMOTE,KMeansSMOTE

plt.style.use('fivethirtyeight')

import seaborn as sns

sns.set()
from joblib import parallel_backend
from joblib.memory import Memory

patch_sklearn()
def allow_stopping(func):
    def wrapper():
        try:
            value = func()
            return value
            # gc.collect()
        except KeyboardInterrupt as e:
            print("Program Stopped")
        gc.collect()

    return wrapper


import warnings

warnings.filterwarnings("ignore")

__refresh__ = 1


def run_if_refresh(func):
    def wrapper():
        global __refresh__
        if __refresh__ == 1:
            value = func()
            return value
        else:
            print(
                "Using Cache, Set Refresh to '__refresh__=1' to regenerate "
                "the output of this  function"
            )

    return wrapper


def gen_train_test(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=10
    )
    return X_train, X_test, y_train, y_test


def categorise_input(X: pd.DataFrame):
    X.loc[:, nominal + ordinal] = X.loc[:, nominal + ordinal].astype("category")
    return X

KAGGLE_ENV = 1
cwd = os.path.abspath(os.getcwd())
if "mlop3n/Pycharm" in cwd:
    KAGGLE_ENV = 0
    
if KAGGLE_ENV==1:
    
    categoriser = FunctionTransformer(categorise_input, feature_names_out="one-to-one")
categoriser = FunctionTransformer(categorise_input, feature_names_out="one-to-one")


def quick_test(X):
    clfs = [
        RandomForestClassifier(class_weight="balanced_subsample", random_state=42),
        DecisionTreeClassifier(class_weight="balanced", random_state=42),
        HistGradientBoostingClassifier(random_state=42),
        LogisticRegressionCV(max_iter=1000, class_weight="balanced", random_state=42),
    ]
    y = raw_data.target
    X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=0.5)
    for clf in clfs:
        y_pred = clf.fit(X_train, y_train).predict(X_test)
        score = f1_score(y_test, y_pred, average="macro")
        print(f"{clf.__class__.__name__} :: {score}")


def find_correlated_features(df, threshold=0.8):
    correlated_features = set()
    correlation_matrix = df.corr()
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    return list(correlated_features)

        
"""
PCA MCA Equivalent
"""

# df = raw_data
def MCA(df):
    tmp_nom = np.zeros(df.shape[0])
    for c in df.columns:
        nom_f = df[c]
        nom_f_vc = nom_f.value_counts()    
        nom_f_ohe = pd.get_dummies(nom_f)
        nom_f_vc_pk = nom_f_vc/df.shape[0]
        for ci in nom_f_ohe.columns:
            nom_f_ohe[ci] = (nom_f_ohe[ci]/nom_f_vc_pk[ci])-1
        tmp_nom = np.c_[tmp_nom,nom_f_ohe.to_numpy()]
        # break
    final_tmp_nom = tmp_nom[:,1:]
    dime = PCA(svd_solver='full',)
    final_nome_t  = dime.fit_transform(final_tmp_nom)
    return final_nome_t

        
"""
Binary Target Distributions
"""

def binary_target_dist(df):
    bin_target_0 = pd.DataFrame(columns=binary, index=[0,1,2])
    bin_target_1 = pd.DataFrame(columns=binary, index=[0,1,2])
    for c in binary:
        target_census = df.target.value_counts().to_dict()    
        f = df.groupby([c])['target'].value_counts().sort_index().unstack().transpose()
        # print(f[0])
        bin_target_0[c] = f[0]
        bin_target_1[c] = f[1]
    for i in range(3):
        bin_target_0.loc[i,:] = bin_target_0.loc[i,:] / target_census[i]
        bin_target_1.loc[i,:] = bin_target_1.loc[i,:] / target_census[i]
    return bin_target_0, bin_target_1

def m_VI(df):
    VI = np.linalg.inv(np.cov(df, rowvar=False))
    return VI

class ColumnSelectors:
    def __init__(self, default=None):
        self.dtype_info = {
            "binary__v_1": "Binary",
            "binary__v_11": "Binary",
            "binary__v_14": "Binary",
            "binary__v_26": "Binary",
            "binary__v_27": "Binary",
            "binary__v_28": "Binary",
            "binary__v_30": "Binary",
            "binary__v_9": "Binary",
            "nominal__v_12": "Nominal",
            "nominal__v_18": "Nominal",
            "nominal__v_20": "Nominal",
            "nominal__v_21": "Nominal",
            "nominal__v_25": "Nominal",
            "nominal__v_3": "Nominal",
            "nominal__v_32": "Nominal",
            "nominal__v_4": "Nominal",
            "ordinal__v_0": "Ordinal",
            "ordinal__v_10": "Ordinal",
            "ordinal__v_13": "Ordinal",
            "ordinal__v_15": "Ordinal",
            "ordinal__v_17": "Ordinal",
            "ordinal__v_19": "Ordinal",
            "ordinal__v_22": "Ordinal",
            "ordinal__v_23": "Ordinal",
            "ordinal__v_24": "Ordinal",
            "ordinal__v_29": "Ordinal",
            "ordinal__v_31": "Ordinal",
            "ordinal__v_33": "Ordinal",
            "ordinal__v_5": "Ordinal",
            "ordinal__v_6": "Ordinal",
            "ratio__v_16": "Ratio",
            "ratio__v_2": "Ratio",
            "ratio__v_34": "Ratio",
            "ratio__v_35": "Ratio",
            "ratio__v_36": "Ratio",
            "ratio__v_37": "Ratio",
            "ratio__v_38": "Ratio",
            "ratio__v_39": "Ratio",
            "ratio__v_40": "Ratio",
            "ratio__v_7": "Ratio",
            "ratio__v_8": "Ratio",
        }

        self.ordinal_cols = [
            i for i in self.dtype_info if self.dtype_info[i] == "Ordinal"
        ]
        self.nominal_cols = [
            i for i in self.dtype_info if self.dtype_info[i] == "Nominal"
        ]
        self.binary_cols = [
            i for i in self.dtype_info if self.dtype_info[i] == "Binary"
        ]
        self.ratio_cols = [i for i in self.dtype_info if self.dtype_info[i] == "Ratio"]
        self.ordinal = make_column_selector(
            pattern="|".join(self.ordinal_cols),
        )
        self.nominal = make_column_selector(
            pattern="|".join(self.nominal_cols),
        )
        self.binary = make_column_selector(
            pattern="|".join(self.binary_cols),
        )
        self.ratio = make_column_selector(
            pattern="|".join(self.ratio_cols),
        )
        self.nominal_ohe = None
        self.ordinal_ohe = None
        
    def add_feature(self,dtype,feature_name):
        dtype_map = {
            'o': self.ordinal_cols,
            'r': self.ratio_cols,
            'b': self.binary_cols,
            'n': self.nominal_cols
        }
        dtype_map[dtype].append(feature_name)
        return dtype_map[dtype]

    def ordinal_selector(self):
        return self.ordinal

    def nominal_selector(self):
        return self.nominal

    def binary_selector(self):
        return self.binary

    def ratio_selector(self):
        return self.ratio
    
    def ordinal_features(self):
        return self.ordinal_cols

    def nominal_features(self):
        return self.nominal_cols

    def binary_features(self):
        return self.binary_cols

    def ratio_features(self):
        return self.ratio_cols
    
    def update_ohe_features(self,n,o):
        self.nominal_ohe = n
        self.ordinal_ohe = o
        
    def nominal_ohe_features(self):
        return self.nominal_ohe
    
    def ordinal_ohe_features(self):
        return self.ordinal_ohe
    

    def categorise_data(self, df: pd.DataFrame = None):
        """
        Categorise Data based on given data
        :params df : pandas.Dataframe
        """
        if isinstance(df, pd.DataFrame):
            ordinal_data = df.loc[:, self.ordinal_cols]
            nominal_data = df.loc[:, self.nominal_cols]
            binary_data = df.loc[:, self.binary_cols]
            ratio_data = df.loc[:, self.ratio_cols]
        else:
            print("Please provide valid Data")
        return ordinal_data, nominal_data, binary_data, ratio_data


column_directory = ColumnSelectors()

DATA_PATH = "/kaggle/input/students-drop-out-prediction/"
DATA_SAVE_PATH = "/kaggle/working/"
TRAIN_DATA = "train.csv"
TEST_DATA = "test.csv"
KAGGLE_ENV = 1
BENCHMARK_INPUT = '/kaggle/input/sub-sample/'
cwd = os.path.abspath(os.getcwd())
if "mlop3n/Pycharm" in cwd or "u164131" in cwd:
    KAGGLE_ENV = 0

if KAGGLE_ENV == 0:
    LOCAL_PATH = "../data"
    DATA_PATH = LOCAL_PATH + DATA_PATH
    DATA_SAVE_PATH = LOCAL_PATH + DATA_SAVE_PATH
    BENCHMARK_INPUT = LOCAL_PATH + BENCHMARK_INPUT

CACHE_ = Memory(DATA_SAVE_PATH,verbose=0)

def write_raw_data(raw_data: pd.DataFrame, raw_data_eval: pd.DataFrame):
    global DATA_SAVE_PATH
    raw_data.to_parquet(DATA_SAVE_PATH + "train.parquet")
    raw_data_eval.to_parquet(DATA_SAVE_PATH + "test.parquet")
    
def write_raw_data_ohe(raw_data_ohe: pd.DataFrame, raw_data_ohe_eval: pd.DataFrame):
    global DATA_SAVE_PATH
    raw_data_ohe.to_parquet(DATA_SAVE_PATH + "train_ohe.parquet")
    raw_data_ohe_eval.to_parquet(DATA_SAVE_PATH + "test_ohe.parquet")

# TODO rerun this script to rename the target column


def reload_raw_data():
    global DATA_SAVE_PATH
    raw_data = pd.read_parquet(DATA_SAVE_PATH + "train.parquet")
    raw_data_eval = pd.read_parquet(DATA_SAVE_PATH + "test.parquet")
    return raw_data, raw_data_eval

def reload_raw_data_ohe():
    global DATA_SAVE_PATH
    raw_data_ohe = pd.read_parquet(DATA_SAVE_PATH + "train_ohe.parquet")
    raw_data_ohe_eval = pd.read_parquet(DATA_SAVE_PATH + "test_ohe.parquet")
    return raw_data_ohe,raw_data_ohe_eval

benchmark = pd.read_csv(BENCHMARK_INPUT+'sample.csv')

def make_submission(y):
    trial=0
    y_df = pd.DataFrame(y, columns=["label"])
    y_df.index.rename('id',inplace=True)
    y_df.to_csv(
        DATA_SAVE_PATH + "submission.csv",
    )
    if trial==0:
        benchmark.to_csv(DATA_SAVE_PATH+ "submission.csv")

def rename_columns_with_dtype(DATA_PATH=DATA_PATH, DATA_SAVE_PATH=DATA_SAVE_PATH, clear_history=True):
    """
    Set clear_history to False
    """
    initial_column_directory = ColumnSelectors()
    train_data = pd.read_csv(DATA_PATH + TRAIN_DATA, index_col=0)
    test_data = pd.read_csv(DATA_PATH + TEST_DATA, index_col=0)

    raw_dtypes_info = {}
    saved_dtypes_info = initial_column_directory.dtype_info
    for k, v in saved_dtypes_info.items():
        tmp = k.split("__")
        data_type = tmp[0]
        column_name = tmp[1]
        raw_dtypes_info[column_name] = k

    train_data.rename(columns=raw_dtypes_info, inplace=True)
    test_data.rename(columns=raw_dtypes_info, inplace=True)
    train_data.rename(columns={"label": "target"}, inplace=True)
    if clear_history:
        write_raw_data(train_data, test_data)
    return train_data, test_data


# rename_columns_with_dtype()
def reset_data(remove_cache=True):
    return rename_columns_with_dtype(clear_history=remove_cache)
    

raw_data, raw_data_eval = reset_data()

X = raw_data.drop(["target"], axis=1)
y = raw_data.target

ordinal = column_directory.ordinal_features()
nominal = column_directory.nominal_features()
binary = column_directory.binary_features()
ratios = column_directory.ratio_features()

ordinal_data, nominal_data, binary_data, ratios_data = column_directory.categorise_data(
    raw_data
)
(
    ordinal_data_eval,
    nominal_data_eval,
    binary_data_eval,
    ratios_data_eval,
) = column_directory.categorise_data(raw_data_eval)

def baseline(raw_data=raw_data, test_size=0.5):
    X = raw_data.drop(["target"], axis=1)
    y = raw_data.target
    # quick_test(X)
    X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=test_size)

    random_forest = BalancedRandomForestClassifier(random_state=10)
    tmp_clf = make_pipeline(
        RFE(
            estimator=BalancedRandomForestClassifier(random_state=42),
            n_features_to_select=50,
        ),
        AdaBoostClassifier(
            base_estimator=random_forest, n_estimators=4, random_state=10
        ),
    )
    with parallel_backend("loky"):
        y_pred = tmp_clf.fit(X_train, y_train).predict(X_test)
        print(classification_report(y_test, y_pred))
        scores = cross_validate(
            tmp_clf,
            X,
            y,
            cv=StratifiedKFold(n_splits=4),
            n_jobs=-1,
            scoring="f1_macro",
            return_train_score=True,
        )
        print("F1_Train", scores["train_score"].mean())
        print("F1_Test", scores["test_score"].mean())


# baseline()

__refresh__ = 1
raw_data, raw_data_eval = reset_data()
def gen_categorical_mask(df):
    categorical_f_mask =[]
    # X = raw_data.columns
    for f in df.columns:
        if f in ordinal+nominal+binary:
            categorical_f_mask.append(True)
        else:
            categorical_f_mask.append(False)
    return categorical_f_mask


def update_ordinal_data(raw_data=raw_data, raw_data_eval=raw_data_eval):
    categorical_columns = [ordinal, nominal]
    ordinal_data = raw_data.loc[:, ordinal].copy()
    ordinal_data_eval = raw_data_eval.loc[:, ordinal].copy()
    raw_data.loc[:, ordinal] = ordinal_data - ordinal_data.min()
    raw_data_eval.loc[:, ordinal] = ordinal_data_eval - ordinal_data.min()
    # update_categorical_data()
    # disp = raw_data.loc[:, ordinal].min().plot(color="r", rot=90)
    # disp = raw_data_eval.loc[:, ordinal].min().plot(rot=90)
    # write_raw_data(raw_data, raw_data_eval)
    return raw_data, raw_data_eval

def sort_columns_by_name(raw_data=raw_data, raw_data_eval=raw_data_eval):
    raw_data = raw_data.sort_index(axis=1)
    raw_data_eval = raw_data_eval.sort_index(axis=1)
    raw_data.to_parquet(DATA_SAVE_PATH + "train.parquet")
    raw_data_eval.to_parquet(DATA_SAVE_PATH + "test.parquet")
    return raw_data, raw_data_eval

# @run_if_refresh
def ordinally_encode_nominal_data(raw_data=raw_data, raw_data_eval=raw_data_eval):
    ordinal_enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=125)
    nominal_enc_data = ordinal_enc.fit_transform(raw_data.loc[:, nominal])
    nominal_enc_data_eval = ordinal_enc.transform(raw_data_eval.loc[:, nominal])
    raw_data.loc[:, nominal] = nominal_enc_data
    raw_data_eval.loc[:, nominal] = nominal_enc_data_eval
    # write_raw_data(raw_data, raw_data_eval)
    return raw_data, raw_data_eval

# @run_if_refresh
def binary_feature_sum(raw_data=raw_data, raw_data_eval=raw_data_eval):
    global column_directory
    raw_data["binary__sum"] = raw_data.loc[:, binary].sum(axis=1)/7
    raw_data_eval["binary__sum"] = raw_data_eval.loc[:, binary].sum(axis=1)/7
    # write_raw_data(raw_data, raw_data_eval)
    column_directory.add_feature('b','binary__sum')
    return raw_data, raw_data_eval

# @run_if_refresh
def standardise_ratio_features(
    raw_data=raw_data.copy(), raw_data_eval=raw_data_eval.copy()
):
    # Load Data
    payload = raw_data.loc[:, ratios]
    payload_eval = raw_data_eval.loc[:, ratios]
    mscaler = MinMaxScaler()
    # Apply Scaling
    payload_t = mscaler.fit_transform(payload)
    payload_eval_t = mscaler.transform(payload_eval)
    # New Column Names
    new_ordinal_col_names = mscaler.get_feature_names_out()
    new_ordinal_col_names = [x.replace(".", "_") for x in new_ordinal_col_names]
    # Create Dataframe with new features to concatenate
    tmp_raw_data = pd.DataFrame(
        payload_t, columns=new_ordinal_col_names, index=raw_data.index
    )
    tmp_raw_data_eval = pd.DataFrame(
        payload_eval_t, columns=new_ordinal_col_names, index=raw_data_eval.index
    )
    # Concatenate in place
    raw_data = pd.concat([tmp_raw_data, raw_data.drop(ratios, axis=1)], axis=1)
    raw_data_eval = pd.concat(
        [tmp_raw_data_eval, raw_data_eval.drop(ratios, axis=1)], axis=1
    )
    # Write modified data to disk
    # write_raw_data(raw_data, raw_data_eval)
    # return raw_data
    return raw_data, raw_data_eval
    
def ordinal_norm(df):
    scaler = PowerTransformer()
    tmp_ord = df.loc[:,ordinal].to_numpy()
    tmp_ord_sq= tmp_ord @ tmp_ord.T 
    tmp_ord_sum = np.sqrt(tmp_ord_sq.sum(axis=1))
    tmp_ord_nom = scaler.fit_transform(tmp_ord_sum.reshape(-1,1))
    return tmp_ord_nom

def gen_ordinal__norm(raw_data=raw_data, raw_data_eval=raw_data_eval):
    global column_directory
    raw_data['ordinal__norm'] = ordinal_norm(raw_data)
    raw_data_eval['ordinal__norm'] = ordinal_norm(raw_data_eval)
    column_directory.add_feature('o','ordinal__norm')
    return raw_data, raw_data_eval

def gen_ohe_data(raw_data=raw_data, raw_data_eval=raw_data_eval):
    global column_directory
    X = pd.concat([raw_data.loc[:,raw_data_eval.columns],raw_data_eval],ignore_index=True,axis=0)
    ohe = OneHotEncoder(sparse=False,dtype=np.int8)
    ohe.fit(X.loc[:,nominal+ordinal])
    nominal_ohe = [x for x in ohe.get_feature_names_out() if 'nominal__' in x]
    ordinal_ohe = [x for x in ohe.get_feature_names_out() if 'ordinal__' in x]
    ohe_data = pd.DataFrame(ohe.transform(raw_data.loc[:,nominal+ordinal]),columns=ohe.get_feature_names_out(),index=raw_data.index)
    ohe_data_eval = pd.DataFrame(ohe.transform(raw_data_eval.loc[:,nominal+ordinal]),columns=ohe.get_feature_names_out(),index=raw_data_eval.index)
    other_than_nominal = [x for x in raw_data_eval.columns if 'nominal__' not in x]
    raw_data_ohe = pd.concat([ohe_data,raw_data.loc[:,other_than_nominal]],axis=1)
    raw_data_ohe_eval = pd.concat([ohe_data_eval,raw_data_eval.loc[:,other_than_nominal]],axis=1)
    column_directory.update_ohe_features(nominal_ohe,ordinal_ohe)
    return raw_data_ohe,raw_data_ohe_eval
    
def gen_binary_literal(row):
    bin_chr = [str(row[b]) for b in binary if 'sum' not in b]
    b_l = ""
    for c in bin_chr:
        b_l+=c
    row.binary__literal = int(b_l,2)/255
    return row

def add_literal_to_data(raw_data, raw_data_eval):
    global column_directory
    b_lit_train = pd.DataFrame(columns=["binary__literal"], index=raw_data.index)
    b_lit_eval  = pd.DataFrame(columns=["binary__literal"], index=raw_data_eval.index)
    raw_data['binary__literal']  = "0"
    raw_data_eval['binary__literal']  = "0"
    raw_data = raw_data.apply(gen_binary_literal,axis=1)
    raw_data_eval= raw_data_eval.apply(gen_binary_literal,axis=1)
    column_directory.add_feature('b','binary__literal')
    return raw_data, raw_data_eval

def standardise_ordinal_data(raw_data,raw_data_eval,raw_data_ohe,raw_data_ohe_eval):
    scaler = sklearn.preprocessing.Normalizer()
    scaler.fit(X.loc[:,ordinal])
    ordinal_data = pd.DataFrame(scaler.transform(raw_data.loc[:,ordinal]),columns=ordinal,index=raw_data.index)
    ordinal_data_eval = pd.DataFrame(scaler.transform(raw_data_eval.loc[:,ordinal]),columns=ordinal,index=raw_data_eval.index)
    
    raw_data.loc[:,ordinal] = ordinal_data
    raw_data_ohe.loc[:,ordinal] = ordinal_data
    raw_data_eval.loc[:,ordinal] = ordinal_data_eval
    raw_data_ohe_eval.loc[:,ordinal] = ordinal_data_eval
    return raw_data,raw_data_eval,raw_data_ohe,raw_data_ohe_eval


def apply_transformations():
    raw_data, raw_data_eval = reset_data()

    raw_data, raw_data_eval = update_ordinal_data(raw_data, raw_data_eval)
    # raw_data, raw_data_eval = reload_raw_data()
    raw_data, raw_data_eval = sort_columns_by_name(raw_data, raw_data_eval)
    # raw_data, raw_data_eval = reload_raw_data()
    raw_data, raw_data_eval = ordinally_encode_nominal_data(raw_data, raw_data_eval)    
    # raw_data, raw_data_eval = reload_raw_data()
    raw_data, raw_data_eval = binary_feature_sum(raw_data, raw_data_eval)
    # raw_data, raw_data_eval = reload_raw_data()
    raw_data, raw_data_eval = standardise_ratio_features(raw_data, raw_data_eval)
    raw_data, raw_data_eval = add_literal_to_data(raw_data, raw_data_eval)
    raw_data.sort_index(axis=1,inplace=True)
    raw_data_eval.sort_index(axis=1,inplace=True)
    def make_int_nominal(df):
        df.loc[:,nominal] = df.loc[:,nominal].astype('int')
        return df

    raw_data = make_int_nominal(raw_data)
    raw_data_eval = make_int_nominal(raw_data_eval)
    
    raw_data_ohe, raw_data_ohe_eval = gen_ohe_data(raw_data, raw_data_eval)
    raw_data,raw_data_eval,raw_data_ohe,raw_data_ohe_eval = standardise_ordinal_data(raw_data,raw_data_eval,raw_data_ohe,raw_data_ohe_eval)
    raw_data, raw_data_eval = gen_ordinal__norm(raw_data, raw_data_eval)
    raw_data_ohe['ordinal__norm'] = raw_data['ordinal__norm']
    raw_data_ohe_eval['ordinal__norm'] = raw_data_eval['ordinal__norm']
    print(raw_data.shape)
    print(raw_data_eval.shape)
    raw_data.sort_index(axis=1,inplace=True)
    raw_data_eval.sort_index(axis=1,inplace=True)
    write_raw_data(raw_data, raw_data_eval)
    write_raw_data_ohe(raw_data_ohe, raw_data_ohe_eval)
    
    
apply_transformations()
raw_data, raw_data_eval = reload_raw_data()

raw_data_ohe, raw_data_ohe_eval = reload_raw_data()


def refresh_data():
    global __refresh__
    __refresh__ = 1
    raw_data, raw_data_eval = reset_data()
    # rename_columns_with_dtype()
    apply_transformations()
    raw_data, raw_data_eval = reload_raw_data()
    __refresh__ = 0
    print("Data Refresh Success")
    return raw_data, raw_data_eval
    
# raw_data, raw_data_eval = refresh_data()

features = raw_data.drop(['target'],axis=1).columns

def create_target_encoding():
    new_column_names = []
    stubs = ["_0","_1","_2"]
    for c in ordinal+nominal:
        for _suffix in stubs:
            new_column_names.append(c+_suffix)
    df_train = pd.DataFrame(index=raw_data.index,columns=new_column_names)
    df_eval = pd.DataFrame(index=raw_data_eval.index,columns=new_column_names)
    target_census = raw_data.target.value_counts().to_dict()    
    for c in ordinal+nominal:
        target_counts = raw_data.groupby(c)['target'].value_counts().unstack(fill_value=0)
        tf: pd.DataFrame = target_counts.loc[:,[0,1,2]]
        target_counts['sum'] = tf.sum(axis=1)
        target_counts['std'] = tf.std(axis=1)
        tf2 = tf.divide((target_counts['sum']),axis='rows')
        target_map = {0:0,1:1,2:2}
        for val in target_map:
            mapping = defaultdict(int,tf2.to_dict()[target_map[val]])
            df_train[c+stubs[val]] = raw_data[c].map(mapping)
            df_eval[c+stubs[val]] = raw_data_eval[c].map(mapping)
    return df_train, df_eval

target_encoded_train, target_encoded_eval  = create_target_encoding()

"""
Refreshing Data Bifurication
"""

ordinal = column_directory.ordinal_features()
nominal = column_directory.nominal_features()
binary = column_directory.binary_features()
ratios = column_directory.ratio_features()
nominal_ohe = column_directory.nominal_ohe_features()
ordinal_ohe = column_directory.ordinal_ohe_features()
ordinal_data, nominal_data, binary_data, ratios_data = column_directory.categorise_data(
    raw_data
)
(
    ordinal_data_eval,
    nominal_data_eval,
    binary_data_eval,
    ratios_data_eval,
) = column_directory.categorise_data(raw_data_eval)
original_ordinal_data, original_ordinal_data_eval = reset_data(remove_cache=False)
o_f = [x for x in original_ordinal_data.columns if 'ordinal' in x]
old_ordinal = original_ordinal_data.loc[:,o_f]
old_ordinal_eval = original_ordinal_data_eval.loc[:,o_f]


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


(3796, 45)
(1628, 44)


In [15]:
%matplotlib widget